In [2]:
pip install seaborn
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
df = pd.read_csv ('data.csv')
print (df)

SyntaxError: invalid syntax (743981363.py, line 1)

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
def depth_plot(ax,df,feat,postfix):
    ax.plot(df[feat],df['Depth'],'b.')
    ax.set_xlim(min(df[feat]),max(df[feat]))
    ax.set_ylim(2800,4200)
    ax.invert_yaxis()
    ax.set_xlabel(feat)
    ax.set_ylabel('Depth')
    #ax.figure.set_size_inches(ax.figure.get_size_inches()[1]/2, ax.figure.get_size_inches()[1])
    fig.tight_layout()
    ax.figure.savefig('depth_'+postfix+'.png')
def freq_plot(ax,df,feat,postfix):
    ax.hist(df[feat],color = "blue")
    ax.set_xlabel(feat)
    fig.tight_layout()
    ax.figure.savefig('hist_'+postfix+'.png')
    
    
feats=['DT','NPHI','RHOB','PEF','RT','SGR']

fig, axis = plt.subplots(2,3)
for i,feat in enumerate(feats):  
    ax=axis[math.floor(i/3), math.floor(i%3)]
    freq_plot(ax,df,feat,'before') 
#plt.show()
#plt.cla()


fig, axis = plt.subplots(2,3)
for i,feat in enumerate(feats):  
    ax=axis[math.floor(i/3), math.floor(i%3)]
    depth_plot(ax,df,feat,'before')#,label,xlabel )  
#plt.show()
#plt.cla()

    


In [ ]:
#Remove outliers
'''
instruction to remove the entire rows when they have outlier and/or null values. Note that if any of these conditions are satisfied, the entire row will be deleted.
DT smaller than 0 || DT greater than 100
NPHI smaller than -0.1 || NPHI greater than 0.45
RHOB smaller than 2.0 || RHOB greater than 3.0
PEF smaller than 0.0 || PEF greater than 7.0
RT smaller than 0.0 || RT greater than 2000
GR smaller than 0 || GR greater than150
'''
df = df.dropna(axis=1)
df = df.drop(df[(df['DT'] < 0.0) | (df['DT'] >100.0)].index)
df = df.drop(df[(df['NPHI'] < -0.1) | (df['NPHI'] >0.45)].index)
df = df.drop(df[(df['RHOB'] < 2.0) | (df['RHOB'] >3.0)].index)
df = df.drop(df[(df['PEF'] < 0.0) | (df['PEF'] >7.0)].index)
df = df.drop(df[(df['RT'] < 0.0) | (df['RT'] >2000)].index)
df = df.drop(df[(df['SGR'] < 0.0) | (df['SGR'] >150.0)].index)



In [ ]:

fig, axis = plt.subplots(2,3)
for i,feat in enumerate(feats): 
    ax=axis[math.floor(i/3), math.floor(i%3)]
    freq_plot(ax,df,feat,'after') 
#plt.show()
#plt.cla()


fig, axis = plt.subplots(2,3)
for i,feat in enumerate(feats):  
    ax=axis[math.floor(i/3), math.floor(i%3)]
    depth_plot(ax,df,feat,'after')#,label,xlabel ) 
#plt.show()
#plt.cla()


In [ ]:

#Scale data between 0-1
new_data = preprocessing.MinMaxScaler().fit_transform(df[feats])
df_new=pd.DataFrame(new_data, columns = feats)
df_new.insert(loc=0, column='Depth', value=df['Depth'])


fig, axis = plt.subplots(2,3)
for i,feat in enumerate(feats):  
    ax=axis[math.floor(i/3), math.floor(i%3)]
    depth_plot(ax,df_new,feat,'_scaled')#,label,xlabel )  
#plt.show()
#plt.cla()


In [ ]:
#########scatter plots##############
def scatter_plot(df,feat1,feat2,col,i):
    ax=axis[math.floor(i/3), math.floor(i%3)]
    ax.scatter(df[feat1],df[feat2],s=1,c=df[col],cmap='hot')
    ax.set_xlabel(feat1, fontsize=15)
    ax.set_ylabel(feat2, fontsize=15)
    fig.tight_layout()
    ax.figure.savefig('colored_by_'+col+'.png')

i=0
fig, axis = plt.subplots(2,3)
scatter_plot(df_new,'PEF','SGR',feat,0)  
scatter_plot(df_new,'SGR','RT',feat,1)
scatter_plot(df_new,'PEF','RHOB',feat,2)
scatter_plot(df_new,'PEF','DT',feat,3)
scatter_plot(df_new,'DT','RHOB',feat,4)
scatter_plot(df_new,'NPHI','RHOB',feat,5)

#plt.show()
#plt.cla()

#scatter_plot(df_new,'NPHI','DT',feat,i++)

In [ ]:
#########Density plots##############

def freq_plot(df,feat1,feat2,i):
    ax=axis[math.floor(i/3), math.floor(i%3)]
    sns.set_palette("Reds")
    res = sns.kdeplot(df_new[feat1],df_new[feat2],shade=True,ax=ax)
    res.set(xlabel=feat1, ylabel=feat2)
    fig.tight_layout()
    ax.figure.savefig('colored_by_freq.png')
    i+=1
    #plt.show()
#plt.cla()
fig, axis = plt.subplots(2,3)
i=0   
freq_plot(df_new,'PEF','SGR',i); i+=1
freq_plot(df_new,'PEF','RHOB',i); i+=1
freq_plot(df_new,'PEF','DT',i); i+=1
freq_plot(df_new,'DT','RHOB',i); i+=1
freq_plot(df_new,'NPHI','RHOB',i); i+=1
freq_plot(df_new,'NPHI','DT',i); i+=1
#plt.cla()


In [ ]:
from sklearn.cluster import KMeans,SpectralClustering,AgglomerativeClustering,DBSCAN
from sklearn.mixture import GaussianMixture,BayesianGaussianMixture
from sklearn.metrics import davies_bouldin_score
db_score=100
best_algo='None'
db_arr=[]
labels=[]

def scatter_plot_cluster(df,feat1,feat2,col,clus_name,i):
    ax=axis[math.floor(i/3), math.floor(i%3)]
    ax.scatter(df[feat1],df[feat2],s=1,c=col,cmap='jet')
    ax.set_xlabel(feat1, fontsize=15)
    ax.set_ylabel(feat2, fontsize=15)
    fig.tight_layout()
    ax.figure.savefig(clus_name+'.png')

cls_algos = ['KMeans','SpectralClustering','AgglomerativeClustering','DBSCAN']
for algo in cls_algos:
    if algo=='DBSCAN':
        cls=DBSCAN(eps=0.05)
    else:
        cls=globals()[algo](n_clusters=4) 
    col = cls.fit_predict(df_new[feats])
    db_new=davies_bouldin_score(df_new[feats],col)
    db_arr.append(db_new)
    if db_new<db_score:
        best_algo=algo
        db_score=db_new
        labels=col
    cl_name=algo
    fig, axis = plt.subplots(2,3)
    i=0   
    scatter_plot_cluster(df_new,'PEF','SGR',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'PEF','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'PEF','DT',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'DT','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'NPHI','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'SGR','RT',col,cl_name,i); i+=1

In [ ]:
cls_algos = ['GaussianMixture','BayesianGaussianMixture']
for algo in cls_algos:
    cls = GaussianMixture(n_components=4)
    col = cls.fit_predict(df_new[feats])
    cl_name=algo
    db_new=davies_bouldin_score(df_new[feats],col)
    db_arr.append(db_new)
    if db_new<db_score:
        best_algo=algo
        db_score=db_new
        labels=col
    fig, axis = plt.subplots(2,3)
    i=0   
    scatter_plot_cluster(df_new,'PEF','SGR',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'PEF','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'PEF','DT',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'DT','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'NPHI','RHOB',col,cl_name,i); i+=1
    scatter_plot_cluster(df_new,'SGR','RT',col,cl_name,i); i+=1

In [ ]:
print(db_arr)
print("best algo is ",best_algo,"with score ",db_score)


In [ ]:
df_new['label']=labels
print(df_new)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

import numpy as np



def plot_regression(y_test,y_pred,title,cls_id,i):
    ax=axis[i]
    ax.scatter(y_test,y_pred,color="black")
    z = np.polyfit(y_test,y_pred, 1)
    p = np.poly1d(z)

    #add trendline to plot
    ax.plot(y_test, p(y_test),color="blue", linewidth=3)
    #plt.plot(np.array(X)[ind_test,:],y_pred, color="blue", linewidth=3)
    ax.set_title(title,fontsize=10)
    fig.tight_layout()
    ax.figure.savefig('Regression_cluster'+str(cls_id)+'.png')
#select a cluster
cluster=df_new[df_new['label']==1].reset_index()
X=cluster[['DT', 'RHOB', 'SGR', 'NPHI']]
y=cluster['PEF']
mseArr=[]



In [ ]:
from sklearn.preprocessing import PolynomialFeatures
PF = PolynomialFeatures(degree=3, include_bias=True)
pFeats=PF.fit_transform(X)
pFeats_y=PF.fit_transform(np.array(y).reshape(-1,1))
X_train, X_test, y_train, y_test,ind_train,ind_test = train_test_split(pFeats, pFeats_y,np.array(range(0,len(y))), test_size=0.33, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
y_pred=reg.predict(X_test)
mseArr.append(MSE(y_test, y_pred))





In [ ]:
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor
from pyGRNN import GRNN
from sklearn.neural_network import MLPRegressor
def apply_regression(reg_mod,X_train, y_train,X_test):
    if reg_mod=='linear':
        reg = LinearRegression()
    elif reg_mod=='lasso':
        reg = Lasso(alpha=0.0005)
    elif reg_mod=='knn':
        reg = KNeighborsRegressor(n_neighbors=3)
    elif reg_mod=='DT':
        reg = regressor = DecisionTreeRegressor(random_state=42)
    elif reg_mod=='GB':
        reg = GradientBoostingRegressor(random_state=42)
    elif reg_mod=='RF':
        reg = RandomForestRegressor(random_state=42)
    elif reg_mod=='ADR':
        reg = AdaBoostRegressor(random_state=42)
    elif reg_mod=='ET':
        reg = ExtraTreesRegressor(random_state=42)
    elif reg_mod=='GRNN':    
        reg = GRNN(calibration = 'gradient_search')
    elif reg_mod=='ANN':    
        reg = MLPRegressor(random_state=42)
    print(reg)
    reg.fit(X_train, y_train)
    y_pred=reg.predict(X_test) 
    return y_pred
regressions=['poly','linear','lasso','knn','DT','GB','RF','ADR','ET','GRNN','ANN']
X_train, X_test, y_train, y_test,ind_train,ind_test = train_test_split(X, y,np.array(range(0,len(y))), test_size=0.33, random_state=42)
reg=None
for reg_mod in regressions:
    reg=None
    if reg_mod=='poly':
        continue
    else:
        print(reg_mod)
        y_pred=apply_regression(reg_mod,X_train, y_train,X_test)
        mseArr.append(MSE(y_test, y_pred))
       

In [ ]:
mseArr=np.array(mseArr)
best_ind=mseArr.argsort()[:4]
print(mseArr)
print(best_ind)
best_algos=[regressions[i] for i in best_ind if i>0 ]
print(best_algos)



In [ ]:
for c in range(0,4):
    fig, axis = plt.subplots(len(best_algos))
    ii=0
    cluster=df_new[df_new['label']==c].reset_index()
    X=cluster[['DT', 'RHOB', 'SGR', 'NPHI']]
    y=cluster['PEF']
    X_train, X_test, y_train, y_test,ind_train,ind_test = train_test_split(X, y,np.array(range(0,len(y))), test_size=0.33, random_state=42)
    for reg_mod in best_algos:
        print(reg_mod)
        y_pred=apply_regression(reg_mod,X_train, y_train,X_test)
        #mseArr.append(MSE(y_test, y_pred))
        plot_regression(y_test,y_pred,reg_mod,c,ii);ii+=1